## PreProcess

In [1]:
"""_summary_
"""
import requests
response = requests.get("http://mirror.coggle.club/seg_risky_training_anno.csv")
# 检查请求是否成功
if response.status_code == 200:
    with open("/home/tipriest/data/seg_risky/seg_risky_training_anno.csv", "wb") as file:
        file.write(response.content)
        print("文件下载成功")
else:
    print("文件下载失败")


文件下载成功


In [42]:
# !apt update > /dev/null; apt install aria2 git-lfs axel -y > /dev/null
# !pip install ultralytics==8.2.0 numpy pandas opencv-python Pillow matplotlib > /dev/null
# !axel -n 12 -a  http://mirror.coggle.club/seg_risky_training_data_01.zip -o ~/data/seg_risky/seg_risky_training_data_01.zip
# !unzip -q /home/tipriest/data/seg_risky/training_data_10.zip -d /home/tipriest/data/seg_risky/


### Import Libs

In [1]:
import os, shutil
import glob
import json
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
"""
这是一个示例模块。

此模块包含与示例功能相关的代码。
"""
BASE_DATA_PATH = "/home/tipriest/data/seg_risky"
# 一共有1020575的数据
training_anno = pd.read_csv(os.path.join(
    BASE_DATA_PATH, "seg_risky_training_anno.csv"))
train_jpgs = []
for i in range(0, 2, 1):
    train_jpgs += [x.replace('/home/tipriest/data/seg_risky/', '')
                    for x in glob.glob(os.path.join(BASE_DATA_PATH, f'{i}/*.jpg'))]
training_anno = training_anno[training_anno['Path'].isin(train_jpgs)]
training_anno['Polygons'] = training_anno['Polygons'].apply(json.loads)


In [3]:
# df = pd.read_csv(os.path.join(
#     BASE_DATA_PATH, "seg_risky_training_anno.csv"))
# df[['Prefix', 'Suffix']] = df['Path'].str.split('/', expand=True)
# prefix_counts= df['Prefix'].value_counts()
# print(prefix_counts)
# for i in range(16):
#     folder_name = format(i, 'x')
#     folder_path = f"folder_{folder_name}"
#     train_jpgs = [x for x in glob.glob(os.path.join(BASE_DATA_PATH, f'{folder_name}/*.jpg'))]
#     print(i, len(train_jpgs))


Prefix
d    64212
3    64160
c    63942
a    63892
f    63881
2    63877
b    63870
6    63808
8    63789
0    63785
7    63725
4    63685
9    63628
5    63507
1    63453
e    63361
Name: count, dtype: int64


## Format dateset

In [3]:
"""
Format Train and Test Set
"""
YOLO_SEG_DATASET_BASE_PATH = os.path.join(BASE_DATA_PATH, "datasetYOLO")
YOLO_SEG_DATASET_TRAIN_PATH = os.path.join(YOLO_SEG_DATASET_BASE_PATH, "train/")
YOLO_SEG_DATASET_VALID_PATH = os.path.join(YOLO_SEG_DATASET_BASE_PATH, "valid/")

if os.path.exists(YOLO_SEG_DATASET_BASE_PATH):
    shutil.rmtree(YOLO_SEG_DATASET_BASE_PATH)
os.makedirs(YOLO_SEG_DATASET_TRAIN_PATH)
os.makedirs(YOLO_SEG_DATASET_VALID_PATH)

def normalize_polygon(polygon, img_width, img_height):
    normalized_polygon = [(min(x, img_width) / img_width, min(y, img_height) / img_height) for x, y in polygon]
    return normalized_polygon

for row in training_anno.iloc[20000:].iterrows():
    shutil.copy(os.path.join(BASE_DATA_PATH, row[1].Path), YOLO_SEG_DATASET_TRAIN_PATH)
    img = cv2.imread(os.path.join(BASE_DATA_PATH, row[1].Path))
    img_height, img_width = img.shape[:2]
    txt_filename = os.path.join(YOLO_SEG_DATASET_TRAIN_PATH + row[1].Path.split('/')[-1][:-4] + '.txt')
    with open(txt_filename, 'w') as up:
        for polygon in row[1].Polygons:
            normalized_polygon = normalize_polygon(polygon, img_width, img_height)
            normalized_coords = ' '.join([f'{coord[0]:.3f} {coord[1]:.3f}' for coord in normalized_polygon])
            up.write(f'0 {normalized_coords}\n')

Premature end of JPEG file


In [4]:
# 制作验证集III
for row in training_anno.iloc[:20000].iterrows():
    shutil.copy(os.path.join(
        BASE_DATA_PATH, row[1].Path), YOLO_SEG_DATASET_VALID_PATH)

    img = cv2.imread(os.path.join(BASE_DATA_PATH, row[1].Path))
    img_height, img_width = img.shape[:2]
    txt_filename = os.path.join(
        YOLO_SEG_DATASET_VALID_PATH + row[1].Path.split('/')[-1][:-4] + '.txt')
    with open(txt_filename, 'w') as up:
        for polygon in row[1].Polygons:
            normalized_polygon = normalize_polygon(
                polygon, img_width, img_height)
            normalized_coords = ' '.join(
                [f'{coord[0]:.3f} {coord[1]:.3f}' for coord in normalized_polygon])
            up.write(f'0 {normalized_coords}\n')


In [5]:
YOLO_SEG_DATASET_BASE_PATH = os.path.join(BASE_DATA_PATH, "datasetYOLO")

In [6]:
with open(os.path.join(os.getcwd(), "cfg/data.yaml"), 'w') as up:
    data_root = os.path.abspath(YOLO_SEG_DATASET_BASE_PATH)
    up.write(f'''
path: {data_root}
train: train
val: valid

names:
    0: alter
''')

## Load Pretrained Model & Train model

In [10]:
# !wget http://mirror.coggle.club/yolo/Arial.ttf -O /home/tipriest/.cache/torch/hub/checkpoints/Arial.ttf
# !wget http://mirror.coggle.club/yolo/yolov8n-v8.2.0.pt -O /home/tipriest/.cache/torch/hub/checkpoints/yolov8n.pt
# !wget http://mirror.coggle.club/yolo/yolov8n-seg-v8.2.0.pt -O /home/tipriest/.cache/torch/hub/checkpoints/yolov8n-seg.pt

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from ultralytics import YOLO
 
if __name__ == '__main__':
    model = YOLO(r'D:\yolo\yolov11\ultralytics-main\datasets\yolo11.yaml')
    model.train(data=r'D:\yolo\yolov11\ultralytics-main\datasets\data.yaml',
                cache=False,
                imgsz=640,
                epochs=100,
                single_cls=True,  # 是否是单类别检测
                batch=8,
                close_mosaic=10,
                workers=0,
                device='0',
                optimizer='SGD',
                amp=True,
                project='runs/train',
                name='exp',
                )

In [ ]:
"""
first train the model
"""
from ultralytics import YOLO
import os
model_base_path = "/home/tipriest/.cache/torch/hub/checkpoints"
# yolov8n-seg、yolov8s-seg、yolov8m-seg、yolov8l-seg和yolov8x-seg，模型的大小依次递增
# model = YOLO(os.path.join(model_base_path, "yolo11x.pt"))
model = YOLO(os.path.join(model_base_path, "yolo11x.pt"))
model.train(data=os.path.join(os.getcwd(), "cfg/data.yaml"), 
        cache=True,
        epochs=50,
        single_cls=True,  # 是否是单类别检测
        batch=15,
        # close_mosaic=10,
        # workers=0,
        device='0',
        optimizer='SGD',
        amp=True,
        project='runs/train',
        # name='exp'
        )


New https://pypi.org/project/ultralytics/8.3.18 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.11 🚀 Python-3.10.14 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4080 SUPER, 15959MiB)
engine/trainer: task=detect, mode=train, model=/home/tipriest/.cache/torch/hub/checkpoints/yolo11x.pt, data=/home/tipriest/Documents/AlgorithmPractice/src/deepLearning/DataWhale/Yolo/cfg/data.yaml, epochs=50, time=None, patience=100, batch=15, imgsz=640, save=True, save_period=-1, cache=True, device=0, workers=8, project=runs/train, name=train2, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=True, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=F

train: Scanning /home/tipriest/data/seg_risky/datasetYOLO/train.cache... 107237 images, 0 backgrounds, 19 corrupt: 100%|██████████| 107238/107238 [00:00<?, ?it/s]

train: WARNING ⚠️ /home/tipriest/data/seg_risky/datasetYOLO/train/036f9c42ccb5342307995f88171f956f.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.002]
train: WARNING ⚠️ /home/tipriest/data/seg_risky/datasetYOLO/train/03a26fcbd2f13b33b63becea41766b01.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.002]
train: WARNING ⚠️ /home/tipriest/data/seg_risky/datasetYOLO/train/044090b5e3ee683ecc47added8b9e2e2.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.01]
train: WARNING ⚠️ /home/tipriest/data/seg_risky/datasetYOLO/train/04c146f2ed1c8fec418a52d995171092.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.002]
train: WARNING ⚠️ /home/tipriest/data/seg_risky/datasetYOLO/train/04ef9b36f47593acfb7d33524e94f542.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.002]
train: WARNING ⚠️ /home/tipriest/data/seg_risky/datasetYOLO/train/0636d

train: 176.4GB RAM required to cache images with 50% safety margin but only 42.8/62.0GB available, not caching images ⚠️


val: Scanning /home/tipriest/data/seg_risky/datasetYOLO/valid.cache... 20000 images, 0 backgrounds, 4 corrupt: 100%|██████████| 20000/20000 [00:00<?, ?it/s]

...

                   all      19996      29149       0.96      0.882      0.943      0.793

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      12/50        15G     0.9298     0.7577      1.073         33        640: 100%|██████████| 7148/7148 [36:37<00:00,  3.25it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 667/667 [02:25<00:00,  4.57it/s]


                   all      19996      29149      0.959      0.888      0.945      0.798

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      13/50        15G       0.92     0.7405      1.066         38        640: 100%|██████████| 7148/7148 [35:52<00:00,  3.32it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 667/667 [02:27<00:00,  4.51it/s]


                   all      19996      29149      0.959      0.892      0.948      0.803


### Continue to train

In [1]:
from ultralytics import YOLO
model = YOLO("./runs/segment/exp3/weights/best.pt")
results = model.train(data=os.path.join(YOLO_SEG_DATASET_BASE_PATH, "data.yaml"), epochs=80, imgsz=512)

/home/tipriest/anaconda3/envs/AIdefense/lib/python3.10/site-packages/ultralytics/nn/tasks.py:732: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_l

FileNotFoundError: [Errno 2] No such file or directory: 'runs/segment/train10/weights/best.pt'

## 训练后需要在这里加载最好的模型

In [2]:
"""
训练之后加载模型
"""
import glob
import os
from ultralytics import YOLO

model = YOLO("./runs/train/exp3/weights/best.pt")
BASE_DATA_PATH = "/home/tipriest/data/seg_risky"
test_imgs = glob.glob(os.path.join(BASE_DATA_PATH, "test_set_A_rename/*/*"))
# test_imgs = glob.glob(os.path.join(base_data_path, "data_set/train/0/*"))
# print(os.path.join(base_data_path, "data_set/train/0/*"))


In [3]:
import numpy as np
from tqdm import tqdm
Polygon = []
for path in tqdm(test_imgs[:]):
    # print(path)
    results = model(path, verbose=False)
    result = results[0]
    if result.masks is None:
        Polygon.append([])
    else:
        image_polygons = []
        for mask in result.masks.xy:
            # 将掩膜的坐标转换为Numpy数组
            mask_coords = np.array(mask)
            # 获取外接矩形(bounding box)
            # print(mask_coords)
            # print()
            if 0 == mask_coords.size:
                continue
            x_min = round(float(np.min(mask_coords[:, 0])), 1)
            x_max = round(float(np.max(mask_coords[:, 0])), 1)
            y_min = round(float(np.min(mask_coords[:, 1])), 1)
            y_max = round(float(np.max(mask_coords[:, 1])), 1)
            bounding_box = [
                [x_min, y_min],
                [x_min, y_max],
                [x_max, y_max],
                [x_max, y_min]
            ]
            image_polygons.append(bounding_box)
            # print(bounding_box)
        Polygon.append(image_polygons)

100%|██████████| 100000/100000 [17:47<00:00, 93.70it/s]


In [4]:
import pandas as pd
submit = pd.DataFrame({
    'Path': [x.split('/')[-1] for x in test_imgs[:]],
    'Polygon': Polygon
})

In [5]:
submit = pd.merge(submit, pd.DataFrame({'Path': [x.split('/')[-1] for x in test_imgs[:]]}), on='Path', how='right')
submit = submit.fillna('[]')
submit.to_csv('track2_submit.csv', index=None)